# 実習05-2 データ前処理を含むモデル作成（実習用）
---
データの読み込みと前処理の手順は以下である。（練習用と同様）
1. CSVファイルを確認し、読み込む。
2. データの内容、性質を理解する。
  *   欠損値の確認と処理
3. 入力と出力、学習用とテスト用に分ける。
4. データの前処理をする。（順番はデータの性質などによって変わるかも）
  *   列の選別
  *   欠損値の処理
  *   数値化またはダミー変数化
  *   正規化
5. データの前処理を行った後に学習や予測を行うために、pipelineを作成する。

# 問題の設定
タイタニック号の乗船者のデータから、死亡か生存かを判定する。

# 実習用データセット
実習用のファイル（titanic.csv）を使用する。練習用よりも行、列ともに多いので注意すること。
## 1. CSVファイルの確認と読み込み
### 1-1. CSVファイルの確認
データをpythonで読み込む前に、簡単に中身を確認しておく。

* 行や列のタイトル
* 行数や列数
* 中身
（各自のPCで実施する）

確認ができたら、CSVファイル（titanic.csv）をGoogle Colabにアップロードする。

### 1-2. CSVファイルの読み込み
read_csvを使用してcsvファイルを読み込む。
```
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv('ファイル名', index_col=0) #最初の列がIDなので行名にする
```

In [1]:
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv('titanic.csv', index_col=0)

In [2]:
#表示して確認してもよい
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


**【課題】実習用のデータは、何行何列のデータですか。**

**【回答】**
891 rows × 11 columns

## 2. データの内容、性質を理解する
各データの性質（数値なのかカテゴリ変数なのか文字列なのか、など）を理解する。

> Survived：死亡or生存（目的変数、二値）<br>
> Pclass：旅客クラス（多値カテゴリ変数）<br>
> Name：名前（文字列）<br>
> Sex：性別（二値カテゴリ変数）<br>
> Age：年齢（数値）<br>
> SibSp：同乗した兄弟、配偶者の数（数値）<br>
>	Parch：同船した両親、子供の数（数値）<br>
>	Ticket：チケット番号<br>
> Fare：運賃（数値）<br>
> Cabin：客室番号 <br>
> Embarked：乗船地<br>


### 欠損値の確認
まずは各列に欠損値があるか確認する。
（欠損値の個数を数える）

In [3]:
#各列の欠損数を確認
df.isnull().sum()

,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0
Cabin,687


※欠損値のある行を削除する場合は、入力と出力を取り出す前に実施する。今回は、行の削除はしないことにして、次に進む。<br>
（行の削除をする場合は、dropnaなどを使用して、入出力に分ける前に削除する。）

## 3. 入力と出力、訓練用とテスト用に分ける
まず、予測したい列（Survived）を出力、それ以外の列を入力として、入力と出力に分ける。
また、train_test_splitを使用して訓練用とテスト用に分割する。（テスト用20%）
```
# 出力の列を抽出
y = df["出力の列"]
# 入力
X = df.drop(["出力の列"], axis=1)
# 訓練用とテスト用に分ける（random_stateは学籍番号にする）
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=割合, random_state=学籍番号, stratify=y)
```

In [31]:
# 入力と出力に分ける。訓練用とテスト用に分ける
y = df["Survived"]
# 入力
X = df.drop(["Survived"], axis=1)

# 訓練用とテスト用に分ける（random_stateは学籍番号にする）
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2220042, stratify=y)

##4. データの前処理
### 4-1. 前処理方針の決定
入力（出力以外）の各列に対して、前処理を決定する。前処理の例には、以下のようなものがある。

*   列の削除（不要、欠損値が多いなど）
*   欠損値の補完
*   数値の正規化
*   数値化またはダミー変数化（数値以外の場合など）
*   そのまま使用

入力のそれぞれの列をどう処理するか検討する。

**【課題】各列の処理をどうするかを決めて記述する。**<br>
* そのまま使用する列、削除する列についてはまとめて記述してよい。
* 方針を決められない場合は、練習用と同じでよいが、残りの列はすべて削除にすること。

### □列名
処理を書く

### □列名、列名、・・・
そのまま使用する

### □列名、列名、・・・
使用しない。（削除）



### 1-4. 前処理を含めたモデル作成、訓練
パイプラインを作成する準備として、列ごとに必要な前処理をColumnTransformerで準備する。
1列に複数の処理を行う場合は、pipelineで記述する。

In [32]:
# 必要な物を先にインポートしてもよい
from sklearn.impute import SimpleImputer  #欠損値の補完
from sklearn.preprocessing import OrdinalEncoder #数値ラベルに変換
from sklearn.preprocessing import OneHotEncoder #ダミー変数に変換
from sklearn.preprocessing import MinMaxScaler #正規化

from sklearn.compose import ColumnTransformer #列ごとに違う処理をする
from sklearn.pipeline import Pipeline # pipelineの作成

# SVM（SVC）を使用する
from sklearn.svm import SVC

In [35]:
# 上記のパイプラインを含めて、各列での前処理をColumnTransformerで作成する
# 例に合わせる場合
# ct = ColumnTransformer([
#     ('drop_col','drop',["Cabin","Name","Ticket","SibSp","Parch","Embarked"]),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'),["Pclass"]),
#     ('enc', OrdinalEncoder(),["Sex"]),
#     ('num', Pipeline([
#         ('imputer', SimpleImputer(strategy='mean')),
#          ('scaler', MinMaxScaler())
#          ]), ['Age']),
#     ],
# remainder='passthrough') # 今回は上で処理していない行は無いので指定不要
ct = ColumnTransformer([
    ('drop_col','drop',["Cabin","Name","Ticket","SibSp","Parch"]),
    ('onehot', OneHotEncoder(handle_unknown='ignore'),["Pclass"]),
    ('enc', OrdinalEncoder(),["Sex"]),
    ('num_age', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
         ('scaler', MinMaxScaler())
         ]), ['Age']),
    ('embarked_enc', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[['S', 'C','Q']]))
    ]), ['Embarked']),
    ],
remainder='passthrough') # 今回は上で処理していない行は無いので指定不要

In [36]:
# fit_transformにx_trainを入力して変換を確認してもよい。
ct.fit_transform(x_train)

array([[ 0.        ,  0.        ,  1.        , ...,  0.13294798,
         1.        , 18.7875    ],
       [ 0.        ,  0.        ,  1.        , ...,  0.23347575,
         0.        ,  7.8958    ],
       [ 0.        ,  0.        ,  1.        , ...,  0.07011812,
         0.        , 12.475     ],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.49736115,
         0.        , 15.75      ],
       [ 0.        ,  0.        ,  1.        , ...,  0.23347575,
         0.        ,  8.05      ],
       [ 1.        ,  0.        ,  0.        , ...,  0.49736115,
         0.        ,  0.        ]])

前処理を行ってからモデルに入力するPipelineを作成する。<br>
とりあえずモデルはSVCでよい。

In [37]:
# 前処理を行ってからモデルに入力するPipelineを作成する
# パイプラインの作成(前処理→モデル)
full_pipeline = Pipeline([
    ('preprocess',ct),
    ('model', SVC()) #モデル
    ],
    verbose=True #パイプラインの実行状況を表示
)

In [38]:
# 訓練用データをfitに入力して学習する
full_pipeline.fit(x_train, y_train)

[Pipeline] ........ (step 1 of 2) Processing preprocess, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_col', 'drop',
                                                  ['Cabin', 'Name', 'Ticket',
                                                   'SibSp', 'Parch']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Pclass']),
                                                 ('enc', OrdinalEncoder(),
                                                  ['Sex']),
                                                 ('num_age',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age']),
                                                 ('embarked_enc',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OrdinalEncoder(categories=[['S',
                                                                                               'C',
                                                                                               'Q']]))]),
                                                  ['Embarked'])])),
                ('model', SVC())],
         verbose=True)

訓練用データとテスト用データのそれぞれに対してscoreを計算する。

In [39]:
# 訓練用データとテスト用データのそれぞれに対してscoreを計算する
full_pipeline.score(x_train, y_train), full_pipeline.score(x_test, y_test)

(0.6797752808988764, 0.6703910614525139)

最後に、テストデータの一部を入力して、予測ができるかどうかを確認しておく。
```
# predictで予測できるか確認する
full_pipeline.predict(x_test[:10]), y_test[:10]
```

In [40]:
# predictで予測できるか確認する
full_pipeline.predict(x_test[:10]), y_test[:10]

(array([1, 0, 0, 0, 0, 1, 1, 0, 0, 0]),
 PassengerId
 830    1
 295    0
 870    1
 621    0
 143    1
 864    0
 803    1
 280    1
 18     1
 626    0
 Name: Survived, dtype: int64)

## 提出物など

実習が終わったら、以下の記入、表示を教員にチェックしてもらうこと。

*   実習用ファイルは何行何列か
*   前処理の方針
*   訓練用とテスト用のscore

このファイルをipynbでダウンロードして提出してください。（ファイル名のクラス番号氏名を確認すること。）